In [1]:
from utils import LoginCredentials
import wandb

authenticator = LoginCredentials()

wandb.login(key=authenticator.wandb_key)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: finnhenri-smidt. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/henrismidt/.netrc


True

In [2]:
import torch
from torchvision import transforms
import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import WandbLogger
from torch.utils.data import DataLoader
import wandb


from dataset import MRIImageDataModule, MRIDataset
from models import MobileViTLightning
from utils import get_best_device
from datetime import datetime


import lightning.pytorch as pl
import torch
import numpy as np
import random

def set_reproducibility(seed=42):
    # Set Python random seed
    random.seed(seed)
    
    # Set Numpy seed
    np.random.seed(seed)
    
    # Set PyTorch seed
    torch.manual_seed(seed)
    
    # If using CUDA:
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    
    # Control sources of nondeterminism
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    # PyTorch Lightning utility to seed everything
    pl.seed_everything(seed, workers=True)

# Example of setting up a reproducible environment
set_reproducibility(42)


Seed set to 42


## Configure Dataloader

In [3]:
from transformers import MobileViTImageProcessor
# Load the preprocessor
model_ckpt = "apple/mobilevit-x-small"
processor = MobileViTImageProcessor.from_pretrained(model_ckpt)

# Load and preprocess the CIFAR-10 dataset
def transform(image):
    # Use MobileViTImageProcessor for preprocessing
    return processor(image, return_tensors="pt")["pixel_values"].squeeze(0)

csv_path = 'Data/metadata_for_preprocessed_files.csv'
data_module = MRIImageDataModule(csv_path, slice_number=70, transform=transform, batch_size=1)
data_module.setup()
train_loader = data_module.train_dataloader()
val_loader = data_module.val_dataloader()

/Users/henrismidt/anaconda3/envs/alzheimer/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
model = MobileViTLightning(model_ckpt=model_ckpt, num_labels=4)
print(type(model)) 
trainer = L.Trainer(max_epochs=5, devices='auto', accelerator='auto')
trainer.fit(model=model, train_dataloaders=train_loader, val_dataloaders=val_loader)


/Users/henrismidt/anaconda3/envs/alzheimer/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of MobileViTForImageClassification were not initialized from the model checkpoint at apple/mobilevit-x-small and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 384]) in the checkpoint and torch.Size([4, 384]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([4]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/

<class 'models.mobilevit_lightning.MobileViTLightning'>


Training: |          | 0/? [00:00<?, ?it/s]

/Users/henrismidt/Documents/Informatik/Master/Alzheimer_Detection/dataset.py:58: PerformanceWarning: indexing past lexsort depth may impact performance.
  rows = self.df.loc[(id, slice_num)]
It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.
/Users/henrismidt/Documents/Informatik/Master/Alzheimer_Detection/dataset.py:47: PerformanceWarning: indexing past lexsort depth may impact performance.
  float_label = self.df.loc[(id, self.slice_number)]['CDR'].iloc[0]
/Users/henrismidt/Documents/Informatik/Master/Alzheimer_Detection/dataset.py:49: PerformanceWarning: indexing past lexsort depth may impact performance.
  age = torch.tensor(self.df.loc[(id, self.slice_number)]['Age']).float()
`Trainer.fit` stopped: `max_epochs=5` reached.


# WandB Sweeps

In [6]:
import os
import torch

# Define the sweep configuration
sweep_config = {
  "method": "grid", 
  "metric": { 
    'name': 'val_loss',
    "goal": "minimize"  
  },
  "parameters": { 
    "epochs": {"values": [10]},
  }
}

# Custom callback for tracking and saving the best model with unique names
class BestModelSaver(L.Callback):
    def __init__(self):
        self.best_val_loss = float('inf')
        self.checkpoint_dir = "model_checkpoints"
        # Use the current timestamp as a unique identifier for each training run
        self.run_id = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

    def on_validation_end(self, trainer, pl_module):
        current_val_loss = trainer.callback_metrics['val_loss'].item()

        if current_val_loss < self.best_val_loss:
            self.best_val_loss = current_val_loss

            os.makedirs(self.checkpoint_dir, exist_ok=True)
            checkpoint_path = os.path.join(self.checkpoint_dir, f"best_model_{self.run_id}_epoch={trainer.current_epoch}.ckpt")

            torch.save(pl_module.state_dict(), checkpoint_path)

            # Optionally, log the model checkpoint in W&B with a unique name
            wandb.log_artifact(checkpoint_path, type='model', name=f'bestModel{trainer.current_epoch}')


# Function to be called for each sweep
def train():
    # Initialize a W&B run
    wandb.init()

    # Access sweep parameters
    config = wandb.config
    epochs = config.epochs

    # Model initialization (modified to use config parameters)
    model = EfficientNetBaseline()
    # Initialize the WandbLogger
    wandb_logger = WandbLogger(project="Baseline", entity="Alzheimer Detection")

    # Best model saver initialization
    best_model_saver = BestModelSaver()

    # Trainer initialization
    trainer = L.Trainer(
        max_epochs=epochs,
    #    logger=wandb_logger,
    #    callbacks=[best_model_saver],  # Add the custom callback
    )

    # Fit the model
    trainer.fit(model=model, train_dataloaders=train_loader, val_dataloaders=val_loader)
    # torch.save(cinn.state_dict(), f'txt_to_img_{hidden_size}_{n_blocks}.pth')

    # Close the W&B run when done
    wandb.finish()


In [7]:
# Initialize sweep
sweep_id = wandb.sweep(sweep=sweep_config, project="Alzheimer Detection", entity="Slice Selection")

# Start sweep
wandb.agent(sweep_id, train, count=9) 


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Create sweep with ID: g1zo2lqu
Sweep URL: https://wandb.ai/network-to-network/cINN-Text-to-Image/sweeps/g1zo2lqu


wandb: Agent Starting Run: mw2x3oc6 with config:
wandb: 	epochs: 40
wandb: 	hidden_size: 128
wandb: 	n_blocks: 15
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
wandb: Currently logged in as: finnhenri-smidt (network-to-network). Use `wandb login --relogin` to force relogin
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELIS

/Users/henrismidt/anaconda3/envs/gnn/lib/python3.9/site-packages/lightning/pytorch/loggers/wandb.py:398: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name                | Type          | Params
------------------------------------------------------
0 | coupling_blocks     | ModuleList    | 1.2 M 
1 | orthogonal_matrices | ParameterList | 57.3 K
------------------------------------------------------
1.2 M     Trainable params
57.3 K    Non-trainable params
1.2 M     Total params
4.918     Total estimated model params size (MB)


Sanity Checking DataLoader 0:  50%|█████     | 1/2 [00:00<00:00, 10.92it/s]

...

Epoch 39: 100%|██████████| 240/240 [00:48<00:00,  4.98it/s, v_num=mxmy]

`Trainer.fit` stopped: `max_epochs=40` reached.


Epoch 39: 100%|██████████| 240/240 [00:48<00:00,  4.96it/s, v_num=mxmy]


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
min_val_loss,█▇▇▅▄▃▂▂▂▁
train_loss,▇▇█▅▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▁▁▁▁▁▁▁▁▁▁▁
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_loss,█▇▆▄▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▄▄
epoch,39
min_val_loss,-1.58059
train_loss,-22.40546
trainer/global_step,9599
val_loss,14.21215


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


In [1]:
import torch
from torchvision import transforms
import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import WandbLogger
from torch.utils.data import DataLoader
from transformers import MobileViTImageProcessor
import wandb

from dataset import MRIImageDataModule, MRIDataset
from models import MobileViTLightning
from utils import get_best_device, LoginCredentials

from datetime import datetime
import lightning.pytorch as pl
import torch
import numpy as np
import random
from sklearn.metrics import f1_score

authenticator = LoginCredentials()
wandb.login(key=authenticator.wandb_key)

def set_reproducibility(seed=42):
    # Set Python random seed
    random.seed(seed)
    
    # Set Numpy seed
    np.random.seed(seed)
    
    # Set PyTorch seed
    torch.manual_seed(seed)
    
    # If using CUDA:
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    
    # Control sources of nondeterminism
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    # PyTorch Lightning utility to seed everything
    pl.seed_everything(seed, workers=True)

# Example of setting up a reproducible environment
set_reproducibility(42)

# Load the preprocessor
model_ckpt = "apple/mobilevit-x-small"
processor = MobileViTImageProcessor.from_pretrained(model_ckpt)

# Load and preprocess the CIFAR-10 dataset
def transform(image):
    # Use MobileViTImageProcessor for preprocessing
    return processor(image, return_tensors="pt")["pixel_values"].squeeze(0)

csv_path = 'Data/metadata_for_preprocessed_files.csv'

# Define sweep configuration
sweep_config = {
    'method': 'grid',
    'parameters': {
        'slice_number': {
            'values': list(range(20, 140, 3))  # Values: 20, 17, 14
        }
    }
}

sweep_id = wandb.sweep(sweep=sweep_config, project="Alzheimer-Detection")

# Define the training function
def train(config=None):
    with wandb.init(config=config):
        config = wandb.config

        data_module = MRIImageDataModule(csv_path, slice_number=config.slice_number, transform=transform, batch_size=48, num_workers=0)
        data_module.setup()
        train_loader = data_module.train_dataloader()
        val_loader = data_module.val_dataloader()
        test_loader = data_module.test_dataloader()

        model = MobileViTLightning(model_ckpt=model_ckpt, num_labels=4)

        wandb_logger = WandbLogger()

        checkpoint_callback = ModelCheckpoint(
            dirpath='model_checkpoints',
            filename=f'slice_numer_{config.slice_number}',
            monitor='val_loss',
            mode='min',
            save_top_k=1
        )

        trainer = L.Trainer(
            max_epochs=20,
            devices='auto',
            accelerator='auto',
            logger=wandb_logger,
            callbacks=[checkpoint_callback],
            log_every_n_steps=24
        )

        trainer.fit(model=model, train_dataloaders=train_loader, val_dataloaders=val_loader)

        # Load best model for testing
        # best_model_path = checkpoint_callback.best_model_path
        # best_model = MobileViTLightning.load_from_checkpoint(best_model_path, model_ckpt=model_ckpt, num_labels=4)

        # # Evaluate on test set
        # best_model.eval()
        # all_preds = []
        # all_labels = []
        # for batch in test_loader:
        #     inputs, labels = batch
        #     outputs = best_model(inputs)
        #     preds = torch.argmax(outputs, dim=1)
        #     all_preds.extend(preds.cpu().numpy())
        #     all_labels.extend(labels.cpu().numpy())

        # f1 = f1_score(all_labels, all_preds, average='weighted')
        # wandb.log({'test_f1_score': f1})

# Run the sweep
wandb.agent(sweep_id, function=train)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: finnhenri-smidt. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/henrismidt/.netrc
Seed set to 42
/Users/henrismidt/anaconda3/envs/alzheimer/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Create sweep with ID: 02iggz9q
Sweep URL: https://wandb.ai/finnhenri-smidt/Alzheimer-Detection/sweeps/02iggz9q


wandb: Agent Starting Run: 4zgxy87x with config:
wandb: 	slice_number: 20
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


/Users/henrismidt/anaconda3/envs/alzheimer/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of MobileViTForImageClassification were not initialized from the model checkpoint at apple/mobilevit-x-small and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 384]) in the checkpoint and torch.Size([4, 384]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([4]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.


epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
train_loss,█▂▁▁▁
trainer/global_step,▁▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▆▆▆▇▇▇▇██
val_loss,█▇▆▅▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁
epoch,19
train_loss,0.60501
trainer/global_step,139
val_loss,0.50798


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
